In [6]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
import os
from moviepy.editor import VideoFileClip
from IPython.display import HTML

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    global prev_first_side
    global prev_second_side
    first_side = []
    first_avg_slope = 0
    first_y_intercept = 0
    second_avg_slope = 0
    second_y_intercept = 0
    second_side = []
    
    min_y = 999
    max_y = -999
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            slope = (y2 - y1)/(x2 - x1)
            y_intercept = y1 - slope * x1
            if slope < 0:
                first_side.append(line)
                first_avg_slope += slope
                first_y_intercept += y_intercept
            else:
                second_side.append(line)
                second_avg_slope += slope
                second_y_intercept += y_intercept
                
            min_y = min(min_y, y1, y2)
            max_y = max(max_y, y1, y2)
    
    if len(first_side) > 0:
        first_avg_slope /= len(first_side)
        first_y_intercept /= len(first_side)
    elif len(prev_first_side) > 0:
        cv2.line(img, (int(prev_first_side[0]), int(prev_first_side[1])), (int(prev_first_side[2]), int(prev_first_side[3])), color, thickness)
        
    if len(second_side) > 0:
        second_avg_slope /= len(second_side)
        second_y_intercept /= len(second_side)
    elif len(prev_second_side) > 0:
        cv2.line(img, (int(prev_second_side[0]), int(prev_second_side[1])), (int(prev_second_side[2]), int(prev_second_side[3])), color, thickness)

        
    for side_line in first_side:
        for x1,y1,x2,y2 in line:
            slope = (y2 - y1)/(x2 - x1)
            # if abs(slope - first_avg_slope) < 0.2:
            min_x = (min_y - first_y_intercept) / first_avg_slope
            max_x = (max_y - first_y_intercept) / first_avg_slope
            prev_first_side = [min_x, min_y, max_x, max_y]
            cv2.line(img, (int(min_x), int(min_y)), (int(max_x), int(max_y)), color, thickness)
                
    for side_line in second_side:
        for x1,y1,x2,y2 in line:
            slope = (y2 - y1)/(x2 - x1)
            # if abs(slope - second_avg_slope) < 0.2:
            min_x = (min_y - second_y_intercept) / second_avg_slope
            max_x = (max_y - second_y_intercept) / second_avg_slope
            prev_second_side = [min_x, min_y, max_x, max_y]
            cv2.line(img, (int(min_x), int(min_y)), (int(max_x), int(max_y)), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

def process_image(image_loaded):
    canny_low_threshold = 150
    canny_high_threshold = 300
    kernel_size = 5
    rho = 2
    theta = np.pi/180
    threshold = 120
    min_line_length = 80
    max_line_gap = 100
    # image_loaded = mpimg.imread(path)
    ysize = image_loaded.shape[0]
    xsize = image_loaded.shape[1]
    vertices = np.array([[(0,ysize),(580, 290), (620, 290), (xsize,ysize)]], dtype=np.int32)
    
    gray = grayscale(image_loaded)
    blur_gray = gaussian_blur(gray, kernel_size)
    edges = canny(blur_gray, canny_low_threshold, canny_high_threshold)
    # print("YSize is", ysize, "and XSize is", xsize)
    image_cut = region_of_interest(edges, vertices)
    image_hough = hough_lines(image_cut, rho, theta, threshold, min_line_length, max_line_gap)
    # image_blank = np.copy(image_loaded)*0
    image_lines = weighted_img(image_hough, image_loaded)
    return image_lines
    # plt.imshow(image_lines)
    
def process_video(video_path, output_path):
    clip = VideoFileClip(video_path)
    processed_clip = clip.fl_image(process_image)
    %time processed_clip.write_videofile(output_path, audio=False)
    return

prev_first_side = []
prev_second_side = []
output_path1 = 'white.mp4'
process_video('solidWhiteRight.mp4', output_path1)
output_path2 = 'yellow.mp4'
process_video('solidYellowLeft.mp4', output_path2)
output_path3 = 'extra.mp4'
process_video('challenge.mp4', output_path3)
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
<video width="960" height="540" controls>
  <source src="{1}">
</video>
<video width="960" height="540" controls>
  <source src="{2}">
</video>
""".format(output_path1, output_path2, output_path3))




[MoviePy] >>>> Building video white.mp4
[MoviePy] Writing video white.mp4


100%|█████████▉| 221/222 [00:02<00:00, 88.83it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: white.mp4 

CPU times: user 1.96 s, sys: 549 ms, total: 2.51 s
Wall time: 2.87 s
[MoviePy] >>>> Building video yellow.mp4
[MoviePy] Writing video yellow.mp4


100%|█████████▉| 681/682 [00:08<00:00, 82.90it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: yellow.mp4 

CPU times: user 6.14 s, sys: 1.64 s, total: 7.78 s
Wall time: 8.62 s
[MoviePy] >>>> Building video extra.mp4
[MoviePy] Writing video extra.mp4


100%|█████████▉| 221/222 [00:02<00:00, 89.23it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: extra.mp4 

CPU times: user 2.04 s, sys: 594 ms, total: 2.63 s
Wall time: 2.89 s
